This is an example showing how to calculate the critical densities of H2O.

In [1]:
%matplotlib inline

import numpy as np
import os
import sys

for std, __std__ in [
    (sys.stdout, sys.__stdout__),
    (sys.stderr, sys.__stderr__),
]:
    # By default Jupyter notebook displays the terminal output.
    # This is for turning of this feature.
    if getattr(std, "_original_stdstream_copy", None) is not None:
        # redirect captured pipe back to original FD
        os.dup2(std._original_stdstream_copy, __std__.fileno())
        std._original_stdstream_copy = None

def lower_keys(p):
    return {k.lower(): p[k] for k in p}

## Import the wrapper

In [2]:
import wrapper_my_radex
wrapper = wrapper_my_radex.myradex_wrapper
about_info = wrapper.about.tobytes()
column_info = wrapper.column_names.tobytes()

In [3]:
column_names = [_.decode() for _ in wrapper.column_names.item().split()]
print([_ for _ in enumerate(column_names)])

[(0, 'iup'), (1, 'ilow'), (2, 'Eup'), (3, 'freq'), (4, 'lam'), (5, 'Tex'), (6, 'tau'), (7, 'Tr'), (8, 'fup'), (9, 'flow'), (10, 'flux_K'), (11, 'flux_int'), (12, 'flux_Jy'), (13, 'beta'), (14, 'Jnu'), (15, 'gup'), (16, 'glow'), (17, 'Aul'), (18, 'Bul'), (19, 'Blu'), (20, 'n_crit'), (21, 'n_crit_old')]


## Configure the wrapper



In [4]:
transition_dir = os.path.expanduser('~/_c/protoplanetary_disk/transitions/')

In [5]:
n_levels, n_item, n_transitions, n_partners = \
    wrapper.config_basic(dir_transition_rates=transition_dir,
                         filename_molecule='oh2o@rovib.dat',
                         tbg=2.73, verbose=True)

In [6]:
print('Number of energy levels', n_levels)
print('Number of transitions', n_transitions)
print('Number of collisional partners', n_partners)

Number of energy levels 411
Number of transitions 7597
Number of collisional partners 2


## Calculate the critical densities

**You must run `run_one_params` once before calling `calc_critical_density`** to feed in the physical parameters.

In [10]:
params = {'Tkin': 300.0,
          'dv_CGS': 1e5,
          'dens_X_CGS': 1e5,
          'Ncol_X_CGS': 1e18,
          'H2_density_CGS': 1e9,
          'HI_density_CGS': 0.0,
          'oH2_density_CGS': 0.0,
          'pH2_densty_CGS': 0.0,
          'HII_density_CGS': 0.0,
          'Electron_density_CGS': 0.0,
          'n_levels': n_levels,
          'n_item': n_item,
          'n_transitions': n_transitions,
          'donotsolve': False,
          'collisioPartnerCrit': 2
         }

"""The keywords to the wrapper function have to be in
lower case, so I have to lower the keys of params.
Of course you can use lower case letters from the beginning."""

params = lower_keys(params)

energies,f_occupations,data_transitions,cooling_rate = wrapper.run_one_params(**params)

In [11]:
for i in range(n_transitions):
    lam_micron = 2.99792458e8*1e6/data_transitions[3,i]
    if 14.5 <= lam_micron <= 15.0:
        print('{:4d}'.format(int(data_transitions[0,i])),
              '{:4d}'.format(int(data_transitions[1,i])),
              '{:8.2f}'.format(data_transitions[2,i]),
              '{:6.2f}'.format(lam_micron),
              '{:10.3e}'.format(data_transitions[5,i]),
              '{:10.3e}'.format(data_transitions[19,i]),
              '{:10.3e}'.format(data_transitions[20,i]))

  40   19  1805.90  14.95  2.577e+02  1.238e+04  4.251e+08
  46   24  2030.99  14.98  2.368e+02  2.860e+02  4.883e+08
  52   30  2288.65  14.91  2.096e+02  4.990e-01  4.974e+08
  65   37  2605.03  14.54  1.728e+02  2.145e+05  4.712e+08
  75   43  2876.11  14.88  1.422e+02  3.410e+03  4.976e+08
  91   50  3232.76  14.51  1.395e+02  1.636e+06  4.652e+08
 106   63  3543.52  14.84  9.836e+01  2.699e-04  2.767e+07
 108   65  3589.98  14.60  1.069e+02  1.284e+00  4.184e+09
 120   78  3867.54  14.74  1.419e+02  2.871e-01  2.323e+11
 126   83  3950.91  14.87  1.313e+02  4.295e+04  4.987e+08
 127   83  3965.86  14.64  1.096e+02  2.516e+01  5.236e+08
 132   90  4178.82  14.73  1.929e+02  4.555e-03  1.318e+14
 136   91  4198.70  14.89  1.123e+02  8.009e+06  5.066e+08
 138   91  4212.65  14.68  9.544e+01  3.934e+04  5.230e+08
 146  104  4438.39  14.92  1.032e+02  8.259e+06  4.815e+08
 173  121  4867.76  14.67  2.773e+02  5.378e-01  1.334e+12
 192  135  5168.69  14.78  2.060e+02  2.111e+06  5.309e+